In [0]:
####  a sample code for word embedding
#!pip install gensim
import re
import nltk
import gensim
from gensim.models import Word2Vec
import numpy as np

nltk.download('stopwords')
from nltk.corpus import stopwords

In [0]:
# importing data
with open('text.txt',encoding="utf8", errors='ignore') as file:
    data = file.read()

# keeping only alphabets
data = re.findall(r'\w+[a-zA-Z]', data.lower())

# removing stopwords
stops = stopwords.words('english')
data = [w for w in data if w not in stops]
data = ' '.join(data)

In [0]:
# total words in one segment
window_size = 6
# inbuilt gensim preprocessing
data_processed = gensim.utils.simple_preprocess(data.lower())
# segmenting words i.e. "window_size" no. of words in one list
data_processed = [data_processed[i:i+window_size] for i in range(0, len(data_processed))]

# skip_gram model in gensim
embedding = Word2Vec(data_processed, size=100, window=window_size, min_count=2, workers=10, sg=1)
# training
embedding_layer = embedding.train(data_processed, total_examples=len(data_processed), epochs=300)

In [0]:
# dataset preparation
# holds input training data
in_data_vectors = np.zeros([len(data_processed), window_size-1, 100])
# holds output label data i.e. input[i] + 1 (next word)
out_data_vectors = np.zeros([len(data_processed), 1, 100])

# storing word vector of each word
for i, sentence in enumerate(data_processed):
    for j in sentence:
        # vectors of first n-1 words will be stored in the input data, where n = window_size (eg: total 6 words, 5 in input, 1 in output)
        if j<(window_size-1):
            in_data_vectors[i][j] = embedding.wv.word_vec(sentence[j])
        else:
            out_data_vectors[i][0] = embedding.wv.word_vec(sentence[j])
            continue

In [0]:
# training and testing set
# no. of training examples at a given percentage
split = int(len(in_data_vectors)*0.8)
x_train = in_data_vectors[0:split]
y_train = out_data_vectors[0:split]

x_test = in_data_vectors[split:len(out_data_vectors)]
y_test = out_data_vectors[split:len(out_data_vectors)]